In [232]:
import pandas as pd
import ast


In [233]:
def expand(column):
    top_data = column['top']
    dict = {item['key'] : item['value'] for item in top_data}
    return pd.Series(dict)



def unstr_to_str_data(path, name):

    data = pd.read_excel(path)

    data.drop(['car_links','new_car_feature'], axis=1, inplace=True)

    # first columns name
    data['new_car_detail'] = data['new_car_detail'].apply(ast.literal_eval)
    new_car_detail_df = pd.json_normalize(data['new_car_detail'])

    # removing the column name and adding the new columns in old data
    data = pd.concat([data.drop(columns=['new_car_detail']), new_car_detail_df], axis=1) # First column over
    
    # second columns
    data['new_car_overview'] = data['new_car_overview'].apply(ast.literal_eval)
    new_car_overview_df = pd.json_normalize(data['new_car_overview'])

    new_columns = new_car_overview_df.apply(expand, axis=1)

    # removing the column top in df and adding the new_columns data
    new_car_overview_df = pd.concat([new_car_overview_df.drop(columns=['top']), new_columns], axis=1)

    # removing the column in old data and adding the df
    data = pd.concat([data.drop(columns=['new_car_overview']), new_car_overview_df], axis=1)

    # third columns

    data['new_car_specs'] = data['new_car_specs'].apply(ast.literal_eval)
    new_car_specs_df = pd.json_normalize(data['new_car_specs'])

    new_columns = new_car_specs_df.apply(expand, axis=1)

    new_car_specs_df = pd.concat([new_car_specs_df.drop(columns=['top']), new_columns], axis=1)

    flattened_data = []

    for car_spec in data['new_car_specs']:

        new_car_specs_data_df = {}

        for category in car_spec['data']:
            for item in category['list']:
                new_car_specs_data_df[item['key']] = item['value']

        flattened_data.append(new_car_specs_data_df)

    df = pd.DataFrame(flattened_data)

    new_car_specs_df = pd.concat([new_car_specs_df.drop(columns=['data']),df], axis=1)

    data = pd.concat([data.drop(columns=['new_car_specs']),new_car_specs_df], axis=1)

    data['City'] = name

    data.to_csv(f"cleaned_{name}_cars.csv")

    return data


In [239]:

path = r"F:\GUVI\Project\Car Dekho\DATA\kolkata_cars.xlsx"
name = "kolkata"

unstr_to_str_data(path, name)

,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,...,Rear Brake Type,Tyre Type,Alloy Wheel Size,No Door Numbers,Ground Clearance Unladen,Cargo Volumn,Compression Ratio,Acceleration,Top Speed,City
0,0,Petrol,Sedan,"70,000",Automatic,3,3rd Owner,Toyota,Toyota Camry,2014,...,Solid Disc,"Tubeless,Radial",17,4,NaN,NaN,NaN,NaN,NaN,kolkata
1,0,Petrol,Hatchback,"23,981",Manual,1,1st Owner,Datsun,Datsun RediGO,2017,...,Drum,NaN,NaN,5,185mm,222,NaN,NaN,NaN,kolkata
2,0,Petrol,SUV,"7,100",Automatic,1,1st Owner,Renault,Renault Kiger,2021,...,Drum,"Tubeless, Radial",NaN,5,NaN,405,NaN,NaN,NaN,kolkata
3,0,Petrol,Hatchback,"71,574",Manual,2,2nd Owner,Hyundai,Hyundai i20,2011,...,Drum,"Tubeless,Radial",14,5,NaN,295 Lit,:1,12.96 Sec,NaN,kolkata
4,0,Diesel,SUV,"50,000",Automatic,2,2nd Owner,Audi,Audi Q3,2014,...,Drum,"Tubeless,Radial",16,5,NaN,460-litres,NaN,8.2 Seconds,212 Kmph,kolkata
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1376,0,Petrol,Hatchback,"10,000",Manual,1,1st Owner,Maruti,Maruti Celerio,2022,...,Drum,"Tubeless, Radial",NaN,5,NaN,313,NaN,NaN,NaN,kolkata
1377,0,Petrol,Hatchback,"1,20,000",Manual,1,1st Owner,Maruti,Maruti Alto 800,2014,...,Drum,Tubeless Tyres,NaN,5,NaN,177-litres,NaN,19 Seconds,140 kmph,kolkata
1378,0,Petrol,Sedan,"50,000",Automatic,3,3rd Owner,Mercedes-Benz,Mercedes-Benz C-Class,2011,...,Solid Disc,"Tubeless,Radial",17,4,NaN,475-litres,9.3:1,8.8 Seconds,230km/hr,kolkata
1379,0,Petrol,Hatchback,"40,000",Manual,1,1st Owner,Maruti,Maruti Ritz,2012,...,Drum,"Tubeless,Radial",14,5,NaN,236-liters,NaN,15 Seconds,156 Kmph,kolkata
